**REDUNDANT?**

In [1]:
source("../../BrusselSprouts/scripts/functions.R")


scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"

initial_timedate=Sys.time()

# what needs to be done is individually, and parallely genotype all crosses, then merge them into a DB, and once call the joint genotyping (once per batch of sequencing, so twice in total)

bam_files=list.files(path="~/data/trd/mapped_reads",pattern="JNRC[0-9]*.bam$",full.names = TRUE)
bam_files=c(bam_files, list.files(path="~/data/trd/mapped_reads",pattern="ChrisC[0-9].bam$",full.names = TRUE))
bam_files=naturalsort(bam_files)
crosses=unlist(lapply(basename(bam_files), getFirst_v2, split="."))


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
for(b in bam_files){
    if(file.exists(paste0(b,".depth.gz"))){
        next
    }
    cmd=paste0("samtools depth ",b," | gzip > ",b,".depth.gz")
    execute_cmd_sbatch(cmd,mem="4G",cpu="1",time="short",env="bwaetc",jobname="SamDepth")
    Sys.sleep(1)
}

In [ ]:
# wait till finished

In [74]:
for(b in bam_files){
    cov=fread(paste0(b,".depth.gz"))

    cov_summary=cov %>%
      group_by(V1, window = cut(V2, breaks = seq(min(V2), max(V2), by = 1000) 
                                , 
                                labels = seq(min(V2), max(V2), by = 1000)[-length(seq(min(V2), max(V2), by = 1000))]
                               )
              ) %>%
      summarise(avg_cov = mean(V3)) %>% filter(!is.na(window))

    cov_summary=mutate(cov_summary, window=as.numeric(as.character(window)))

    summary(cov_summary)

    colnames(cov_summary)[1]="chr"
    cov_summary=cov_summary[naturalorder(cov_summary$chr),]

    chrs=summarise(group_by(cov_summary, chr),maxpos=max(window)+1000)
    chrs=chrs[naturalorder(chrs$chr),]

    cov_summary$global_pos=cov_summary$window
        for(c in 2:length(unique(chrs$chr))){
            chr=unique(chrs$chr)[c]
            cov_summary$global_pos[cov_summary$chr==chr]=cov_summary$window[cov_summary$chr==chr]+sum(chrs$maxpos[chrs$chr%in%unique(cov_summary$chr)[1:(c-1)]])
    }
    
    fwrite(cov_summary, paste0(b,".depth.gz.summary.gz"))
    
    
    }

`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.
